In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import warnings
import tensorflow as tf
import joblib
import time
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# --- Configuration ---

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.get_logger().setLevel('ERROR')

# Define Paths (Relative)
INPUT_FILE = 'Input_data_CNN.csv'
SAVED_MODELS_DIR = "Saved_models"
TMP_DIR = "tmp"
CNN_SAVE_PATH = os.path.join(SAVED_MODELS_DIR, "CNN_model")
GBC_SAVE_PATH = os.path.join(SAVED_MODELS_DIR, "gradient_boosting_model_best.pkl")

# Create directories if they do not exist
if not os.path.exists(SAVED_MODELS_DIR):
    os.makedirs(SAVED_MODELS_DIR)
if not os.path.exists(TMP_DIR):
    os.makedirs(TMP_DIR)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("- Libraries imported and directories configured.")

- Libraries imported and directories configured.


In [2]:
# --- CNN Data Constants ---
CNN_WINDOW_SIZE = 6
LABELS = ['enter', 'leave', 'stay', 'out']
FEATURE_COLUMNS = [f'RSSI{i}' for i in range(1, 11)]
TARGET_COLUMN = 'state'

def load_and_process_cnn_data(csv_path):
    """
    Loads data from CSV and creates sliding windows for CNN input.
    """
    if not os.path.exists(csv_path):
        print(f"- Error: File {csv_path} not found.")
        return None, None

    df = pd.read_csv(csv_path)
    
    # Basic cleaning check
    if len(df) < CNN_WINDOW_SIZE:
        print("- Error: Not enough data.")
        return None, None
        
    all_X = []
    all_y = []
    
    # Apply sliding window across the continuous dataset
    # Note: We are not grouping by day here based on the provided file structure,
    # but assuming a continuous stream for training.
    for i in range(len(df) - CNN_WINDOW_SIZE + 1):
        # Label is the state at the last row of the window
        label_value = df.iloc[i + CNN_WINDOW_SIZE - 1][TARGET_COLUMN]
        
        if pd.isna(label_value) or label_value not in LABELS:
            continue
            
        # Extract features (RSSI 1-10)
        window_features = df.iloc[i : i + CNN_WINDOW_SIZE][FEATURE_COLUMNS].to_numpy()
        
        # Normalize: (RSSI + 100) / 100
        normalized_window = (window_features + 100.0) / 100.0
        
        # Reshape for CNN: (6, 10, 1)
        reshaped_window = normalized_window.reshape(CNN_WINDOW_SIZE, 10, 1)
        
        # One-hot encode label
        label = np.zeros(len(LABELS))
        label[LABELS.index(label_value)] = 1
        
        all_X.append(reshaped_window)
        all_y.append(label)
        
    return np.array(all_X), np.array(all_y)

# Load Data
print("- Loading and processing data for CNN...")
X_cnn_all, y_cnn_all = load_and_process_cnn_data(INPUT_FILE)

# Split into Train/Test for CNN Training
# Using random split as per original code logic for CNN training
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(
    X_cnn_all, y_cnn_all, test_size=0.2, random_state=42
)

print(f"- CNN Data Loaded. Total samples: {len(X_cnn_all)}")
print(f"  X_train shape: {X_train_cnn.shape}")
print(f"  y_train shape: {y_train_cnn.shape}")

- Loading and processing data for CNN...
- CNN Data Loaded. Total samples: 12088
  X_train shape: (9670, 6, 10, 1)
  y_train shape: (9670, 4)


In [3]:
# --- CNN Hyperparameters ---
NN_CONN2D_1 = 64
NN_CONN2D_2 = 128
NN_NH_1 = 256
LEARNING_RATE = 0.0001
BATCH_SIZE = 100
SAVE_STEP_EPOCHS = 3000 # As per requirement
DISPLAY_STEP = 100

class MyCNNModel(Model):
    def __init__(self):
        super(MyCNNModel, self).__init__()
        self.conv1 = Conv2D(NN_CONN2D_1, 3, activation='relu', padding='same')
        self.pool1 = MaxPooling2D(2)
        self.conv2 = Conv2D(NN_CONN2D_2, 2, activation='relu')
        self.pool2 = MaxPooling2D(1)
        self.flatten = Flatten()
        self.d1 = Dense(NN_NH_1, activation='relu')
        self.dropout = Dropout(0.5)
        self.d2 = Dense(len(LABELS))

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        if training:
            x = self.dropout(x, training=training)
        return self.d2(x)

# Initialize Model
cnn_model = MyCNNModel()
# Build to print summary
cnn_model.build(input_shape=(None, CNN_WINDOW_SIZE, 10, 1))
print("- CNN Model initialized.")

- CNN Model initialized.


In [4]:
# --- Training Setup ---
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = cnn_model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, cnn_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, cnn_model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# --- Training Loop ---
print(f"- Starting CNN Training for {SAVE_STEP_EPOCHS} epochs...")
total_batch = int(len(X_train_cnn) / BATCH_SIZE)

start_time = time.time()

for epoch in range(SAVE_STEP_EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    # Simple shuffle
    indices = np.arange(len(X_train_cnn))
    np.random.shuffle(indices)
    X_shuffled = X_train_cnn[indices]
    y_shuffled = y_train_cnn[indices]
    
    for batch in range(total_batch):
        start_idx = batch * BATCH_SIZE
        end_idx = (batch + 1) * BATCH_SIZE
        batch_xs = X_shuffled[start_idx:end_idx]
        batch_ys = y_shuffled[start_idx:end_idx]
        
        train_step(batch_xs, batch_ys)
        
    if (epoch + 1) % DISPLAY_STEP == 0:
        print(f"  Epoch {epoch+1}, Loss: {train_loss.result():.4f}, Accuracy: {train_accuracy.result()*100:.2f}%")

print(f"- CNN Training finished in {time.time() - start_time:.2f} seconds.")

# --- Save CNN Model ---
# Saving to 'Saved_models/CNN_model'
cnn_model.save(CNN_SAVE_PATH, save_format="tf")
print(f"- CNN Model saved to: {CNN_SAVE_PATH}")

- Starting CNN Training for 3000 epochs...
  Epoch 100, Loss: 0.4786, Accuracy: 83.38%
  Epoch 200, Loss: 0.3677, Accuracy: 87.30%
  Epoch 300, Loss: 0.2896, Accuracy: 90.20%
  Epoch 400, Loss: 0.2450, Accuracy: 91.91%
  Epoch 500, Loss: 0.2208, Accuracy: 92.54%
  Epoch 600, Loss: 0.2088, Accuracy: 93.01%
  Epoch 700, Loss: 0.2029, Accuracy: 93.34%
  Epoch 800, Loss: 0.1977, Accuracy: 93.36%
  Epoch 900, Loss: 0.1952, Accuracy: 93.40%
  Epoch 1000, Loss: 0.1903, Accuracy: 93.52%
  Epoch 1100, Loss: 0.1894, Accuracy: 93.60%
  Epoch 1200, Loss: 0.1864, Accuracy: 93.72%
  Epoch 1300, Loss: 0.1864, Accuracy: 93.67%
  Epoch 1400, Loss: 0.1860, Accuracy: 93.67%
  Epoch 1500, Loss: 0.1852, Accuracy: 93.69%
  Epoch 1600, Loss: 0.1840, Accuracy: 93.78%
  Epoch 1700, Loss: 0.1830, Accuracy: 93.80%
  Epoch 1800, Loss: 0.1819, Accuracy: 93.83%
  Epoch 1900, Loss: 0.1817, Accuracy: 93.86%
  Epoch 2000, Loss: 0.1816, Accuracy: 93.80%
  Epoch 2100, Loss: 0.1829, Accuracy: 93.79%
  Epoch 2200, Loss: 0

- CNN Model saved to: Saved_models\CNN_model


In [5]:
print("- Generating input features for GBC model using trained CNN...")

# 1. Get CNN predictions for the ENTIRE dataset (to maintain sequence)
# We use X_cnn_all which preserves the sliding window order from the CSV
cnn_predictions_logits = cnn_model.predict(X_cnn_all, verbose=0)

# 2. Create Sliding Windows of CNN Outputs for GBC
# GBC requires a window of 7 consecutive CNN outputs
GBC_WINDOW = 7
gbc_features = []
gbc_labels = []

# Determine labels (convert one-hot back to index)
y_indices = np.argmax(y_cnn_all, axis=1)

# Loop through the CNN predictions
for i in range(len(cnn_predictions_logits) - GBC_WINDOW + 1):
    # Extract window of 7 CNN predictions
    # Shape: (7, 4) -> Flatten to (28,)
    window_preds = cnn_predictions_logits[i : i + GBC_WINDOW].flatten()
    
    # The label for this sequence is the label of the last item in the window (offset by GBC_WINDOW - 1)
    # Note: y_indices[i] corresponds to the label of cnn_predictions[i]
    # So we want the label corresponding to the end of this GBC window
    target_label = y_indices[i + GBC_WINDOW - 1]
    
    gbc_features.append(window_preds)
    gbc_labels.append(target_label)

X_gbc = np.array(gbc_features)
y_gbc = np.array(gbc_labels)

print("- GBC Features generated.")
print(f"  X_gbc shape: {X_gbc.shape}")
print(f"  y_gbc shape: {y_gbc.shape}")

# Split GBC data into Train/Test
# We split this randomly as the GBC model (unlike the input generation) does not rely on sequence during training itself
X_train_gbc, X_test_gbc, y_train_gbc, y_test_gbc = train_test_split(
    X_gbc, y_gbc, test_size=0.2, random_state=42, stratify=y_gbc
)

print(f"  GBC Train samples: {len(X_train_gbc)}")
print(f"  GBC Test samples: {len(X_test_gbc)}")

- Generating input features for GBC model using trained CNN...
- GBC Features generated.
  X_gbc shape: (12082, 28)
  y_gbc shape: (12082,)
  GBC Train samples: 9665
  GBC Test samples: 2417


In [6]:
print("- Starting GBC Training with Grid Search...")

# --- GBC Configuration ---
gradient_boosting = GradientBoostingClassifier()

# Parameter Grid (from original code)
param_grid = {
    'n_estimators': [70, 80, 85, 86, 90],
    'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.45],
    'max_depth': [2, 3, 4, 5, 6]
}

# Stratified K-Fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search
grid_search = GridSearchCV(gradient_boosting, param_grid, cv=5, n_jobs=-1)

start_time_gbc = time.time()

# Fit model
grid_search.fit(X_train_gbc, y_train_gbc)

print(f"- GBC Training finished in {time.time() - start_time_gbc:.2f} seconds.")
print(f"  Best parameters: {grid_search.best_params_}")

- Starting GBC Training with Grid Search...
- GBC Training finished in 1169.92 seconds.
  Best parameters: {'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 86}


In [7]:
print("- Evaluating GBC Model...")

# Predict on Test Set
y_pred_gbc = grid_search.predict(X_test_gbc)

# Classification Report
print("  Gradient Boosting Classification Report:")
print(classification_report(y_test_gbc, y_pred_gbc, target_names=LABELS))

# Save Best Model
joblib.dump(grid_search.best_estimator_, GBC_SAVE_PATH)
print(f"- GBC Model saved to: {GBC_SAVE_PATH}")

print("- All tasks completed successfully.")

- Evaluating GBC Model...
  Gradient Boosting Classification Report:
              precision    recall  f1-score   support

       enter       0.92      0.74      0.82       315
       leave       0.98      0.96      0.97       410
        stay       0.99      0.96      0.97      1113
         out       0.83      0.98      0.90       579

    accuracy                           0.94      2417
   macro avg       0.93      0.91      0.92      2417
weighted avg       0.94      0.94      0.93      2417

- GBC Model saved to: Saved_models\gradient_boosting_model_best.pkl
- All tasks completed successfully.
